In [1]:
import pandas as pd
import csv

In [8]:
data = pd.read_csv("middleman/middleman_original.csv")
data = data.to_numpy()

print(data[0])

[438866 'e0a2c4b3011179c618450577388ec29a6807e517'
 'e0a2c4b3011179c618450577388ec29a6807e517#4d3c73b1a02a59416b246e87c58931d21c438e08#12494dc2c35df54c26fff91d34acea0f428c2b2f#ecea540ec783ccff6682ed23437042b7a96f3f94#4d2e0d8a7d4956675913f5e4946c521c70fe447d#dba2cbd0ee630dc95dffa71309b4ffc6060462b5#79ded54e9cc031a07410e86574eaccd7571e6240#98b2b293c19c06e009e6fed827cc399e16441e28#21388f21039cfe36737e1bfc2d91063fc998ec48#20ae42dcc09ae3a5b295518f3cb4e67abc296532#7e20616fe623da4cddf42b5b2db8dfb49ff884d2#c0ba3e2946ace0b50890c49788cd47feaafd4897#d553ee653b48ecc917f1157cb70655ee05b3847f#cfd976e2ed6d783a2914edae3ec9de91a7a8a719#96bdd46552741c34f4322987e47c243be653d48d#559b42abb545b3e3fc946349ccf9c7bb6b42cd80#b73ff3100f26dbe878f7af11f8d2dea65d4e7a9e'
 '2011-12-22 22:49:29' 'middleman/middleman' 33 False nan nan nan]


In [9]:
# combine same build
unique_build = []

prev_build = data[0][0]
prev_commit = data[0][2]
prev_time = data[0][3]
test_string = str(data[0][9])
tests_run = [data[0][6]]

for i in range(1, len(data)+1):
    if i == len(data):
        if_test_run = False
        for item in tests_run:
            if item is True:
                if_test_run = True
                break
        
        unique_build.append([prev_build, prev_commit, prev_time, if_test_run, test_string])
        break
    
    if data[i][0] == prev_build:
        test_string = test_string + "#" + str(data[i][9])
        tests_run.append(data[i][6])
    else:
        if_test_run = False
        for item in tests_run:
            if item is True:
                if_test_run = True
                break
                
        unique_build.append([prev_build, prev_commit, prev_time, if_test_run, test_string])
        prev_build = data[i][0]
        prev_commit = data[i][2]
        prev_time = data[i][3]
        test_string = str(data[i][9])
        tests_run = [data[i][6]]

# clear same tests
for i in range(len(unique_build)):
    temp_tests = []
    
    for item in unique_build[i][4].split("#"):
        if item !="nan":
            if item not in temp_tests:
                temp_tests.append(item)
    
    write_tests = ""
    for t in temp_tests:
        if write_tests == "":
            write_tests = write_tests + t
        else:
            write_tests = write_tests + "#" + t
        
    unique_build[i][4] = write_tests

for row in unique_build:
    print(row)

[438866, 'e0a2c4b3011179c618450577388ec29a6807e517#4d3c73b1a02a59416b246e87c58931d21c438e08#12494dc2c35df54c26fff91d34acea0f428c2b2f#ecea540ec783ccff6682ed23437042b7a96f3f94#4d2e0d8a7d4956675913f5e4946c521c70fe447d#dba2cbd0ee630dc95dffa71309b4ffc6060462b5#79ded54e9cc031a07410e86574eaccd7571e6240#98b2b293c19c06e009e6fed827cc399e16441e28#21388f21039cfe36737e1bfc2d91063fc998ec48#20ae42dcc09ae3a5b295518f3cb4e67abc296532#7e20616fe623da4cddf42b5b2db8dfb49ff884d2#c0ba3e2946ace0b50890c49788cd47feaafd4897#d553ee653b48ecc917f1157cb70655ee05b3847f#cfd976e2ed6d783a2914edae3ec9de91a7a8a719#96bdd46552741c34f4322987e47c243be653d48d#559b42abb545b3e3fc946349ccf9c7bb6b42cd80#b73ff3100f26dbe878f7af11f8d2dea65d4e7a9e', '2011-12-22 22:49:29', False, '']
[438872, 'e0a2c4b3011179c618450577388ec29a6807e517#4d3c73b1a02a59416b246e87c58931d21c438e08#12494dc2c35df54c26fff91d34acea0f428c2b2f#ecea540ec783ccff6682ed23437042b7a96f3f94#4d2e0d8a7d4956675913f5e4946c521c70fe447d#dba2cbd0ee630dc95dffa71309b4ffc6060462b5#7

[61100164, '08c3969eb35ea657cee76a83f85a4622b7a52c62', '2015-05-04 04:40:33', True, '']
[61173347, 'de5234818dfff89e5d67a54cdc00a1f5c8291096', '2015-05-04 16:52:06', True, '']
[61173355, 'de5234818dfff89e5d67a54cdc00a1f5c8291096', '2015-05-04 17:10:35', True, '']
[61176135, 'ee0f9f00f59fdeb4878073a117457e99b541f0b7', '2015-05-04 17:25:10', True, 'features/asset_hash.feature#features/automatic_directory_matcher.feature#features/builder.feature#features/chained_templates.feature#features/clean_build.feature#features/coffee-script.feature#features/custom-source.feature#features/custom_layout_engines.feature#features/directory_index.feature#features/dynamic_pages.feature#features/extensionless_text_files.feature#features/front-matter-neighbor.feature#features/frontmatter_page_settings.feature#features/gzip.feature#features/helpers_auto_javascript_include_tag.feature#features/helpers_auto_stylesheet_link_tag.feature#features/i18n_builder.feature#features/ignore.feature#features/more-clean_b

In [10]:
# scrap file data from Github
import csv
import requests
from bs4 import BeautifulSoup as bs
import random
import time

with open("middleman/middleman_final.csv", "w", newline='', encoding="utf-8") as f:
    csv_writer = csv.writer(f, delimiter=',')
    csv_writer.writerow(['build ID', 'commit', 'prev commit', 'time', 'file change', 'is test run', 'tests'])
    
    for row in unique_build:
        build_id = row[0]
        commit_id = row[1]
        build_time = row[2]
        is_test_build = row[3]
        test_info = row[4]
        
        temp_file_list = []
        prev_commit = []
        for index in range(len(commit_id.split("#"))):
            url = "https://github.com/middleman/middleman/commit/" + commit_id.split("#")[index]
            print(url)

            r = requests.get(url)
            soup = bs(r.content, "html.parser")

            myDiv = soup.findAll('div', {'class': 'file-info flex-auto min-width-0 mb-md-0 mb-2'})

            for div in myDiv:
                temp_f = div.find('a').text
                
                if temp_f not in temp_file_list:
                    temp_file_list.append(temp_f)
            
            if index == len(commit_id.split("#")) - 1:
                myDiv1 = soup.findAll('div', {'class': 'flex-auto no-wrap text-lg-right text-left overflow-x-auto'})
                
                for div in myDiv1:
                    temp_c = div.findAll('a', {'class': 'sha'})

                    for c in temp_c:
                        temp_prev_c = c.text
                        prev_commit.append(temp_prev_c)
            
        output_file_string = ""
        for f in temp_file_list:
            if output_file_string == "":
                output_file_string = output_file_string + f
            else:
                output_file_string = output_file_string + "#" + f
        print(output_file_string)
        
        output_prev_commit_string = ""
        for c in prev_commit:
            if output_prev_commit_string == "":
                output_prev_commit_string = output_prev_commit_string + c
            else:
                output_prev_commit_string = output_prev_commit_string + "#" + c
        print(output_prev_commit_string)
        
        csv_writer.writerow([build_id, commit_id, output_prev_commit_string, build_time, output_file_string, is_test_build, test_info])
        print(" ")
        
        rand_int = random.randint(1, 4)
        time.sleep(rand_int)

https://github.com/middleman/middleman/commit/e0a2c4b3011179c618450577388ec29a6807e517
https://github.com/middleman/middleman/commit/4d3c73b1a02a59416b246e87c58931d21c438e08
https://github.com/middleman/middleman/commit/12494dc2c35df54c26fff91d34acea0f428c2b2f
https://github.com/middleman/middleman/commit/ecea540ec783ccff6682ed23437042b7a96f3f94
https://github.com/middleman/middleman/commit/4d2e0d8a7d4956675913f5e4946c521c70fe447d
https://github.com/middleman/middleman/commit/dba2cbd0ee630dc95dffa71309b4ffc6060462b5
https://github.com/middleman/middleman/commit/79ded54e9cc031a07410e86574eaccd7571e6240
https://github.com/middleman/middleman/commit/98b2b293c19c06e009e6fed827cc399e16441e28
https://github.com/middleman/middleman/commit/21388f21039cfe36737e1bfc2d91063fc998ec48
https://github.com/middleman/middleman/commit/20ae42dcc09ae3a5b295518f3cb4e67abc296532
https://github.com/middleman/middleman/commit/7e20616fe623da4cddf42b5b2db8dfb49ff884d2
https://github.com/middleman/middleman/comm

https://github.com/middleman/middleman/commit/dd8391be9399d86e01c5ab7b529c40e260a4375e
.travis.yml#lib/middleman.rb#lib/middleman/base.rb#lib/middleman/guard.rb
d04a8b5
 
https://github.com/middleman/middleman/commit/2e0467650eec39acd9495ea3e86969888e271eba
features/sitemap_traversal.features#fixtures/traversal-app/config.rb#fixtures/traversal-app/source/directory-indexed.html.erb#fixtures/traversal-app/source/directory-indexed/sibling.html.erb#fixtures/traversal-app/source/directory-indexed/sibling2.html.erb#fixtures/traversal-app/source/directory-indexed/sub2/index.html.erb#fixtures/traversal-app/source/directory-indexed/sub3/deep.html.erb#fixtures/traversal-app/source/layout.erb#lib/middleman/sitemap/page.rb
dd8391b#cf6661f
 
https://github.com/middleman/middleman/commit/2904054cebc94f0c94af3fc59b682fb45009c56b
middleman-x86-mingw32.gemspec#middleman.gemspec
2e04676
 
https://github.com/middleman/middleman/commit/7e86ed058ffc417bd05b3e3ea1c7f82024f8d6cb
https://github.com/middleman/

https://github.com/middleman/middleman/commit/4b5174f0c81ff30b5f51cd8c355b62ae145ff3da
.yardopts#Rakefile#gem_rake_helper.rb#middleman-core/lib/middleman-core/version.rb
ca10d10
 
https://github.com/middleman/middleman/commit/4422efc270197ca77ff0100edf53e7af6972f05a
gem_rake_helper.rb
4b5174f
 
https://github.com/middleman/middleman/commit/e496d228b05decc03f2e5611e2aeecb8a60f2416
https://github.com/middleman/middleman/commit/347eb2c6b8efa35ea4f04165d48fde578b5d9a4d
middleman-core/lib/middleman-core/core_extensions/rendering.rb#middleman-core/lib/middleman-core/guard.rb#middleman-x86-mingw32.gemspec#middleman/Rakefile#middleman/features/support/env.rb#middleman-more/lib/middleman-more.rb#middleman-more/lib/middleman-more/renderers/sass.rb
4422efc
 
https://github.com/middleman/middleman/commit/50b9ad7b28cbf0fd39f16375aad71050a54daf61
https://github.com/middleman/middleman/commit/aee8e325249ecb965947c63fc3acfdae2f7253c9
middleman-core/lib/middleman-core/core_extensions/front_matter.rb#mi

https://github.com/middleman/middleman/commit/c473181167e3cbcfbfb08c100334b1f451c01814
https://github.com/middleman/middleman/commit/0ac9c0b66246e59c911e3e349393bc622a7dd869
CHANGELOG.md#middleman-core/lib/middleman-core/extensions/directory_indexes.rb#middleman-core/features/frontmatter_page_settings.feature#middleman-core/lib/middleman-core/core_extensions/front_matter.rb#middleman-core/lib/middleman-core/core_extensions/sitemap.rb#middleman-core/lib/middleman-core/cli/build.rb#middleman-core/lib/middleman-core/sitemap/page.rb
56e0c17
 
https://github.com/middleman/middleman/commit/34e40cbe01d02f66055ba5ec596737707113db2b
middleman-core/features/frontmatter_page_settings.feature#middleman-core/lib/middleman-core/core_extensions/front_matter.rb
7282475
 
https://github.com/middleman/middleman/commit/d0dd60fb4bf23edeb71a7b2d27e9f4faf4492879
middleman-core/lib/middleman-core/cli/build.rb#middleman-core/lib/middleman-core/core_extensions/front_matter.rb
34e40cb
 
https://github.com/middl

https://github.com/middleman/middleman/commit/740d4a913f9915c5871330b5113b3959a8f5c40d
middleman-more/features/compass-sprites.feature
5842bf8
 
https://github.com/middleman/middleman/commit/3cabbcf5a4df0b2810ebc8859bf198db0991eb1a
.travis.yml
740d4a9
 
https://github.com/middleman/middleman/commit/4e4e835bf5cea15cf6f345cc6f4fd80f9569ada0
CHANGELOG.md#middleman-core/lib/middleman-core/watcher.rb
3cabbcf
 
https://github.com/middleman/middleman/commit/5b2381f6eb21c2fef86d714e217e8234d7a6d5e8
.travis.yml
4e4e835
 
https://github.com/middleman/middleman/commit/d69ae55fa810338eaee97c06f038dc0ed5e94646
README.md
5b2381f
 
https://github.com/middleman/middleman/commit/3c78d9f171e44806165bf2a5c710b376a23bcd7b
middleman-more/features/sprockets_gems.feature#middleman-more/fixtures/sprockets-app/source/library/js/vendored_include.js#middleman-more/fixtures/sprockets-app/vendor/assets/javascripts/vendored_js.js#middleman-more/lib/middleman-more/core_extensions/sprockets.rb
d69ae55#7458aca
 
https

...

https://github.com/middleman/middleman/commit/7efa5e9dd9bae62ac5f0e2a96af9fc07f2dd844d
middleman-core/lib/middleman-core/util.rb#middleman-core/middleman-core.gemspec
5586784
 
https://github.com/middleman/middleman/commit/8bacc35bf2d71809f58c303f185d3c989dec5c72
middleman-core/lib/middleman-core/core_extensions/data.rb
5586784#6812700
 
https://github.com/middleman/middleman/commit/4e8273295bc4fb9d5f22958ce270c75c0167578c
https://github.com/middleman/middleman/commit/92d96f6604249a662b5cd685bb8c48c567bf6790
.rubocop.yml#middleman-core/features/asset_hash.feature#middleman-core/lib/middleman-core/renderers/sass.rb#middleman-core/lib/middleman-core/util.rb#middleman-core/lib/middleman-core/sources/source_watcher.rb
8bacc35
 
https://github.com/middleman/middleman/commit/e0818e2118d286270c1244470526d12c3235499f
middleman-core/lib/middleman-core/core_extensions/front_matter.rb
4e82732
 
https://github.com/middleman/middleman/commit/b8c0fd34e70d9fab4fb4844798d403c8be614eb7
middleman-cli/li

https://github.com/middleman/middleman/commit/e36fd3f47667ef4e3070a2b2cc012b5815eedc79
CONTRIBUTING.md → .github/CONTRIBUTING.md#.gitignore
eaac03a
 
https://github.com/middleman/middleman/commit/b695f6da1eebae08d4d3b6f2afcdebf2027eed88
middleman-core/lib/middleman-core/application.rb
e36fd3f
 
https://github.com/middleman/middleman/commit/edb1a53060abdc3589cbcfb6a60b69796462c999
CHANGELOG.md#middleman-core/features/page-id.feature#middleman-core/fixtures/page-id-app/config.rb#middleman-core/fixtures/page-id-app/source/fm.html.erb#middleman-core/fixtures/page-id-app/source/index.html.erb#middleman-core/fixtures/page-id-app/source/overwrites/from-default.html.erb#middleman-core/fixtures/page-id-app/source/overwrites/from-frontmatter.html.erb#middleman-core/lib/middleman-core/core_extensions/routing.rb#middleman-core/lib/middleman-core/sitemap/extensions/proxies.rb#middleman-core/lib/middleman-core/sitemap/resource.rb#middleman-core/lib/middleman-core/sitemap/store.rb#middleman-core/lib/

https://github.com/middleman/middleman/commit/b950af89ac448368d1d64ed9003b1998cd48d823
middleman-core/lib/middleman-core/version.rb
5a2f9c8
 
https://github.com/middleman/middleman/commit/b950af89ac448368d1d64ed9003b1998cd48d823
middleman-core/lib/middleman-core/version.rb
5a2f9c8
 
https://github.com/middleman/middleman/commit/9e774b608a00f963bd1c535ed411ad8dff10b513
middleman-core/lib/middleman-core/preview_server.rb
b950af8
 
https://github.com/middleman/middleman/commit/55230229f6a15fbb1adebfc7ab18f64274ac3b52
middleman-core/lib/middleman-core/extensions/external_pipeline.rb
b950af8
 
https://github.com/middleman/middleman/commit/7ae8e904a60b1ffa66e7356b01ea7b66ab544ee7
middleman-core/lib/middleman-core/preview_server.rb
5523022#9e774b6
 
https://github.com/middleman/middleman/commit/3cbd9fdedefb5ce61674ef491d3c492e4e9c9255
middleman-core/lib/middleman-core/template_context.rb
7ae8e90
 
https://github.com/middleman/middleman/commit/49da6156a0e4f87f35454c812d3e8da1d3c75075
middleman

https://github.com/middleman/middleman/commit/2b0f720850ca80e9f28392112d80a47939582a93
middleman-core/lib/middleman-core/template_context.rb
b386dcd
 
https://github.com/middleman/middleman/commit/a14934e08b001802978f53b6bde02adbbf1c206f
README.md
2b0f720#a95f721
 
https://github.com/middleman/middleman/commit/0f2bc1e0ea65bd6bd1e5e859eb4e4daafbdc8307
middleman-core/lib/middleman-core/config_context.rb#middleman-core/lib/middleman-core/core_extensions/front_matter.rb#middleman-core/lib/middleman-core/core_extensions/inline_url_rewriter.rb#middleman-core/lib/middleman-core/extension.rb#middleman-core/lib/middleman-core/extensions/asset_hash.rb#middleman-core/lib/middleman-core/extensions/asset_host.rb#middleman-core/lib/middleman-core/extensions/minify_css.rb#middleman-core/lib/middleman-core/extensions/minify_javascript.rb#middleman-core/lib/middleman-core/extensions/relative_assets.rb#middleman-core/lib/middleman-core/file_renderer.rb#middleman-core/lib/middleman-core/sitemap/resource.

https://github.com/middleman/middleman/commit/3ebd902ec3afb3bd72e640bc846df8686eaaddf2
middleman-core/middleman-core.gemspec
158c3e9
 
https://github.com/middleman/middleman/commit/599cf1e6d32b57d779b65f68fd07f097725cec7c
middleman-cli/lib/middleman-cli/init.rb
3ebd902
 
https://github.com/middleman/middleman/commit/cf4d40caffefced8d7333cc4e207b05e4bfd2cb4
middleman-core/lib/middleman-core/sitemap/resource.rb
599cf1e
 
https://github.com/middleman/middleman/commit/1c57626445e27e35621242e9b8f5e30977e3af96
middleman-core/lib/middleman-core/sitemap/store.rb
cf4d40c
 
https://github.com/middleman/middleman/commit/3e4187568bc363ab9d3fb37bea0afcbe09cf598e
middleman-core/lib/middleman-core/extensions/minify_css.rb#middleman-core/lib/middleman-core/extensions/minify_javascript.rb
1c57626
 
https://github.com/middleman/middleman/commit/849fc65260bb7d25c137e67dd6de637e93bf789c
CHANGELOG.md#middleman-core/lib/middleman-core/version.rb
3e41875
 
https://github.com/middleman/middleman/commit/ccc1cc